# SemCore dataset

In this notebook we'll break down how the SemCore dataset is organised, and see if it will come in useful

In [11]:
import pandas as pd
import xml.etree.ElementTree as ET

In [12]:
semcore_xml = "Data/Raw/semcor.data.xml"

In [ ]:
tree = ET.parse(semcore_xml)
root = tree.getroot()

In [18]:
print(f'{root.tag=}')
print(f'{root.attrib=}')

root.tag='corpus'
root.attrib={'lang': 'en', 'source': 'semcor'}


In [20]:
i = 0
for child in root:
    print(f'{child.tag=}', f'{child.attrib=}')
    i = i + 1
    if i == 10:
        break

child.tag='text' child.attrib={'id': 'd000', 'source': 'br-e30'}
child.tag='text' child.attrib={'id': 'd001', 'source': 'br-l15'}
child.tag='text' child.attrib={'id': 'd002', 'source': 'br-f16'}
child.tag='text' child.attrib={'id': 'd003', 'source': 'br-j42'}
child.tag='text' child.attrib={'id': 'd004', 'source': 'br-g18'}
child.tag='text' child.attrib={'id': 'd005', 'source': 'br-e26'}
child.tag='text' child.attrib={'id': 'd006', 'source': 'br-f18'}
child.tag='text' child.attrib={'id': 'd007', 'source': 'br-f24'}
child.tag='text' child.attrib={'id': 'd008', 'source': 'br-n17'}
child.tag='text' child.attrib={'id': 'd009', 'source': 'br-h17'}
